# Programowanie obiektowe w języku *C#*

## Wstęp

Klasa to pojemnik na dane i metody. Może posiadać:

* nazwę,
* pola,
* właściwości,
* zdarzenia,
* konstruktory/destruktory,
* metody (w tym przedefiniowane operatory np. dodawania).

*Nazwa* to identyfikator typu klasy. W momencie tworzenia instancji klasy nazwa ta jest przekazywana za słowem kluczowym `new`. Pole to zmienna, która przynależy do klasy i zawiera dane, które są procesowane przez funkcje. *Właściwości* podobnie jak pola zawierają dane, ale mogą zawierać dodatkowe fragmenty kodu. Mogą posiadać dodatkowe modyfikatory dostępu. *Zdarzenia* pozwalają użytkownikowi podłączyć się do powiadomień, które klasa wyzwala. Używane są głównie w programowaniu zdarzeniowym oraz programowania `GUI`. *Konstruktor* i *destruktor* zawierają instrukcje, które zostaną wykonane w momencie kolejno, tworzenia i zwalniania instancji klasy. Zwykle *konstruktor* zawiera fragmenty kodu odpowiedzialne za przydział zasobów i inicjalizację zmiennych, a *destruktor* zwalnianie zasobów niezarządzanych (które nie są zarządzane przez `garbage collector`). *Metody* to funkcje, które są własnością klasy.

Poniższy listing przedstawia przykładową deklarację i użycie klasy: 

In [ ]:
public class SimpleClass //nazwa
{
   public int Field; //pole
   //public readonly Field; //tylko do odczytu oprócz konstruktora
   public int Property { get; set; } //właściwość
   public int PropWithDefault { get; set; } = 12; //właściwość zainicjowana
   public int PropWithInit { get; init; } //tylko do odczytu oprócz wyrażenia tworzącego instancję obiektu
   public SimpleClass(){} //konstruktor
   ~SimpleClass(){} //destruktor
   public void DoSomething(int field) //funkcja klasy
   {
      Field = field;
   }
}

SimpleClass instance1 = new SimpleClass(); //nowa instancja
var instance2 = new SimpleClass(); //nowa instancja 2
//SimpleClass instance1 = new(); //nowa instancja 3 (C# 9)

Słowo kluczowe `init` oznacza tutaj, że tylko w wyrażeniu, które tworzy obiekt, można przypisać zmiennej wartość.

Należy oddzielić deklarację klasy od jej instancji. Klasa to byt logiczny, jego fizyczną reprezentację stanowi instancja, która znajduje się w pamięci komputera.

## Funkcje i ich argumenty

Argumenty do funkcji można przekazywać na wiele sposób. W języku C# domyślnie argumenty przekazywane są przez wartość. Wewnątrz funkcji każdy taki obiekt jest kopią zmiennej, która jest przekazywana do funkcji. W przypadku typów prostych (value type jak int, float, double) zmiana wartości takiej zmiennej wewnątrz funkcji nie powoduje zmiany przekazywanej funkcji. Jest to prawidłowe zachowanie, gdyż zgodnie z dobrymi praktykami, funkcja powinna przetwarzać dane, na podstawie parametrów przekazywanych jako argumenty. Ilustruje to poniższy przykład.

In [1]:
public void PassValueType(int argument) {
    argument = argument * 4;
    Console.WriteLine($"PassValueType argument: {argument}");
}

var argument = 1;

PassValueType(argument);

//Console.WriteLine($"Argument: {argument}");

The below script needs to be able to find the current output cell; this is an easy method to get it.

PassValueType argument: 4


Powyższy problem można rozwiązać na trzy sposoby. 

1. Funkcja może zwracać zmodyfikowany argument.
2. Jeśli funkcja zmienia argument oznacza to, że może zostać ona wydzielona jako pole lub właściwość całej klasy, a nie zmienna lokalna.
3. Można dodać słowo kluczowe **`ref`** w ten sposób mówiąc kompilatorowi, że ma przekazać adres zmiennej, a nie tworzyć jej kopii.

Użycie słowa kluczowego **`ref`** ma dwa zastosowania, jedno popularne (dla typów prostych), drugie mniej znane (dla typów referencyjnych jak klasa). Najpierw zostanie omówione pierwsze zastosowanie, które naprawia powyższy problem z przekazywaniem kopii zmiennej.

In [1]:
public void PassValueTypeByRef(ref int argument) {
    argument = 2;
    Console.WriteLine($"PassValueTypeRef argument: {argument}");
}

var argument = 1;

PassValueTypeByRef(ref argument);

Console.WriteLine($"Argument: {argument}");

PassValueTypeRef argument: 2


Argument: 2


Jak widać na powyższym przykładzie, słowo kluczowe **`ref`** należy użyć w dwóch miejscach, w momencie wywoływania funkcji i jej deklaracji. 

Drugim mniej istotnym, a co za tym idzie mniej popularnym zastosowaniem jest inicjalizacja klasy. Bez słowa kluczowego **`ref`**, kompilator przekaże kopię zmiennej wskaźnikowej na typ. Problem pojawia się w momencie inicjalizacji klasy i przekazania adresu do utworzonej kopii zmiennej wewnątrz funkcji. Ilustruje to poniższy kod.

In [1]:
public class ClassContainer 
{
    public int Value { get; set; }
}

public void PassRefType(ClassContainer container) 
{
    container.Value = 2;
    Console.WriteLine($"PassRefType argument: {container.Value}");

    container = new ClassContainer() { Value = 3 };
    Console.WriteLine($"PassRefType argument: {container.Value}");
}

var container = new ClassContainer() { Value = 1 };

Console.WriteLine($"Argument before call: {container.Value}");

PassRefType(container);

Console.WriteLine($"Argument after call: {container.Value}");

Argument before call: 1


PassRefType argument: 2


PassRefType argument: 3


Argument after call: 2


Jak widać na powyższym listingu, w momencie wywołania funkcji powstaje kopia wskaźnika (referencji) do obiektu utworzonego przed wywołaniem funkcji `PassRefType`. Wewnątrz tej funkcji, zmiana wartości zmiennej `value` powoduje zmianę globalnie, co łatwo zaobserwować po tym co zostało wydrukowane na ekranie po wywołaniu funkcji (ostatnia linia kodu). Wracając jednak do funkcji `PassRefType`, po drugiej inicjalizacji obiektu klasy `ClassContainer` wskażnik `container` zawiera już inny adres, niż początkowo. Jednak zmienia się kopia wskaźnika widoczna w przestrzeni nazw funkcji `PassRefType`, a nie wskaźnik `container` zainicjowany jako pierwszy. Problem ten jednak łatwo naprawić przy użyciu **`ref`**, który powoduje przekazanie wskaźnika na wskaźnik, co przedstawia poniższy kod.

In [1]:
public class ClassContainer 
{
    public int Value { get; set; }
}

public void PassRefType(ref ClassContainer container) 
{
    container.Value = 2;
    Console.WriteLine($"PassRefType argument: {container.Value}");

    container = new ClassContainer() { Value = 3 };
    Console.WriteLine($"PassRefType argument: {container.Value}");
}

var container = new ClassContainer() { Value = 1 };

Console.WriteLine($"Argument before call: {container.Value}");

PassRefType(ref container);

Console.WriteLine($"Argument after call: {container.Value}");

Argument before call: 1


PassRefType argument: 2


PassRefType argument: 3


Argument after call: 3


Zmienna container wskazuje na inny obszar pamięci, co łatwo zaobserwować po ostatnim wydruku na ekran wartości zmiennej `value`.

Kolejnym słowem kluczowym związanym z przekazywaniem argumentów do funkcji jest **`out`**, który wymusz przekazanie wprost wartości dla zmiennej z tym słowem kluczowym w ciele wywoływanej funkcji.

In [ ]:
public void PassWithOut(out string name) {
    name = "Ala";
}

string name = null;

PassWithOut(out name);

Słowo kluczowe **`out`** musi pojawić się w dwóch miejscach, w deklaracji funkcji i jej wywołaniu. Nieprzekazanie tego słowa kluczowego w jednym z miejsc powoduje błąd kompilacji.

Dobrą praktyką w przypadku bardziej złożonych funkcji jest w deklaracji funkcji ustawienie parametrów domyślnych. Poniżej znajduje się przykładowa deklaracja tego typu parametrów.

In [1]:
public void PassWithDefaults(string label, int n = 1, int length = 2) {
    for (int i = 0; i < n; i++) {
        Console.WriteLine(label.Substring(0, length));
    }    
}

PassWithDefaults("Hello World", length: 5);

Hello


Do funkcji `PassWithDefaults` został przekazany łańcuch znaków "Hello World" wraz z drugim domyślnym parametrem `length`. Minimalnie do funkcji należy przekazać parametr `label`. Pozostałe parametry są opcjonalne z wartościami domyślnymi.

Ostatnim sposobem na przekazywanie parametrów jest użycie słowa kluczowego **`params`**, które umożliwia agregowanie argumentów w tablice, dzięki czemu w trakcie wywołania funkcji można przekazywać dowolnie wiele argumentów po przecinku.

In [1]:
public void PassWithParams(string label, params int[] indexes) {
    foreach(var index in indexes) {
        Console.WriteLine(label[index]);
    }
}

PassWithParams("Hello World", 0, 1, 2, 3, 4);

H


e


l


l


o


## Przeciążenie funkcji

Jedną z cech języka C# jest możliwość przeciążenia funkcji (oraz konstruktorów). Oznacza to, że wiele zdefiniowanych funkcji ma takie same nazwy, ale różne argumenty lub co najmniej różne typu argumentów. Wywołanie funkcji musi być jednoznaczne, dlatego w przypadku niejednoznaczności kompilator zwróci błąd.

In [ ]:
public void Log(string message) {    
    Console.WriteLine(message);
}

public void Log(Exception exception) {
    Console.WriteLine(exception.Message);
}

public void Log(Exception exception, string customPrefix) {
    Console.WriteLine($"{customPrefix}: {exception.Message}");
}

Log("Message");
Log(new Exception("Message from exception"));
Log(new Exception("Message from exception"), "Prefix");

Message
Message from exception
Prefix: Message from exception


## Enkapsulacja (hermetyzacja)

Oznacza ukrywanie funkcji lub zmiennych klasy, aby były dostępne tylko dla obiektów dziedziczących oraz ogranicza dostępu z zewnątrz. Dostępne są następujące modyfikatory dostępu:

* `public`.
* `private`.
* `protected`.
* `internal`.
* `protected internal`.
* `private protected (C# 7.2)`

Poniższa tabela przedstawia modyfikatory oraz ich ograniczenia w dostępie do ich zasobów.

|Modyfikator|Klasy pochodne z tego samego modułu|Pozostałe klas z tego samego modułu (projektu)|Klasy pochodne z innych modułów|Klasy z innych modułów|
|:-|:-:|:-:|:-:|:-:|
|`private`              |Nie|Nie|Nie|Nie|
|`public`               |Tak|Tak|Tak|Tak|
|`protected`            |Tak|Nie|Tak|Nie|
|`internal`             |Tak|Tak|Nie|Nie|
|`protected internal`   |Tak|Tak|Tak|Nie|
|`private protected`    |Tak|Nie|Nie|Nie|

Modyfikator `protected internal` zawiera sumę logiczną `protected` i `internal`. Z kolei `private protected` umożliwia dostęp tylko klasą pochodnym w ramach tego samego modułu.

Przykładowe użycie dowolnego modyfikator może wygląda następująco:

In [ ]:
public class SimpleClass
{
   private int Field; 
   public int Property { get; protected set; }
   public SimpleClass(){} //konstruktor
   ~SimpleClass(){} //destruktor
   public void DoSomething(int field)
   {
      Field = field;
   }
}

## Dziedziczenie

Język C# umożliwia dziedziczenie jednokrotne (dla klas) i wielokrotne dla interfejsów. Dostępne są również konstrukcje języka `base` (odwołanie do instancji bazowej) i `this` (odwołanie do bieżącej instancji). Dziedziczenie powoduje tworzenie klasy szczegółowej na podstawie ogólnej klasy bazowej. Poniżej znajduje się przykład dziedziczenia:

In [ ]:
public class SpecificClass: SimpleClass
{
   private int Field2; 
   
   public SpecificClass()
   : base() 
   {
      base.Property = 2;
      this.Field2 = 2;
   }
}

Dziedziczymy pola, metody, właściwości i zdarzenia. W zależności od modyfikatora dostępu do tych elementów, w klasie pochodnej możemy z nich skorzystać lub nie.

## Polimorfizm

Pozwala przeciążać metody bazowe umożliwiając ich uszczegółowienie lub zmianę znaczenia. Przykładowo:

In [1]:
public class A
{
   public virtual void Method1() { }
}
public class B : A
{
   public override void Method1() 
   { 
      base.Method1();//metoda bazowa
   }
}		

## Abstrakcja

Poprzez abstrakcję należy rozumieć pewien interfejs, który musi być zaimplementowany w klasie potomnej. Gwarantuje to występowanie metod i właściwości w klasach potomnych. Interfejs to szablon i nie może posiadać instancji. Podobnie jak klasa abstrakcyjna, która może oprócz interfejsu posiadać implementację niektórych metod. Może również wymuszać na obiektach dziedziczących implementacje metod bądź pól.

In [1]:
public interface SimpleInterface
{
   Int32 Property { get; set; }
   void Method1();        
}

public abstract class SimpleAbstract
{
   Int32 Property { get; set; }
   public abstract void Method1();

   public void Method2()
   {
   }
}

## Typy dynamiczne

Umożliwiają w czasie wykonania aplikacji tworzenie klasy. Należy odróżnić słowo kluczowe `var` od deklaracji zmiennej dynamicznej `dynamic`. W pierwszym przypadku kompilator przekaże za nas typ. Jest to skrócony zapis konkretnego typu znanego w czasie kompilacji. W drugim przypadku typ tworzony jest w czasie działania programu. Zbyt częste użycie typu dynamicznego może spowolnić działanie programu. Dodatkowo typ `dynamic` może być użyty jako typ parametru funkcji.

In [1]:
dynamic dynClass = new
{
   Prop1 = 12,
   Prop2 = "Test"
};

## Static

Wszystkie elementy klasy mogą być statyczne. Statyczność klasy powoduje, że istnieje tylko jedna instancja. Tworzeniem klas statycznych zajmuje się środowisko wykonawcze platformy `.NET`. Innym sposobem na tworzenie tylko jednej instancji klasy jest wzorzec projektowy `singleton`:

In [1]:
public class JustOne
{
    private static Single singleInstance;
    private JustOne() {} //konstruktor prywatny    
    public static JustOne GetInstance() 
    {
      singleInstance = singleInstance ?? new JustOne();
      return singleInstance;
    }
}

Error: [object Object]

Operator `??` jest operatorem binarnym. W przypadku, gdy jego lewy argument ma wartość `null` wykonuje instrukcje zawarte w prawym argumencie.

Klasy statyczne mogę też zastępować typ wyliczeniowy `enum` np.:

In [1]:
public static class PersonType
{
    public static readonly string Teacher = "T";
    public static readonly string Student = "S";
}

## Zdarzenia

Zdarzenia deklarujemy podobnie jak pola klasy. Służą do wywoływania metod do obsługi tych zdarzeń z klas zewnętrznych. Przykładowo klasa realizująca zapis do pliku może wywołać zdarzenie `Complete`. Poniższy listing przedstawia obsługę zdarzenia:

In [ ]:
public class Writer
{
  public EventHandler<bool> Complete;

  public void Write(string fileName)
  {
    OnComplete(true);
  }

  public void OnComplete(bool status)
  {
    if (Complete != null)
      Complete(this, status);
    //lub prosciej
    Complete?.Invoke(this, status);
  }
}

static void Main(string[] args)
{
    var writer = new Writer();
    writer.Complete += (source, status) =>
        Console.WriteLine("Write: {0}", status);
    
    writer.Complete += AnotherHandler;

    writer.Write("file.txt");
}
static void AnotherHandler(object source, bool status)
{
	Console.WriteLine("Write: {0}", status);
}

Wewnętrznie dodanie metody obsługi do zdarzenia realizowane jest podobnie do właściwości z tą różnicą, że zdarzenie posiada metody `add()` i `remove()`. Zamiast użycia klasy `EventHandler` można stworzyć własny szablon tzw. deletegat. Tworzenie delegata ilustruje poniższy listing:

In [1]:
delegate void SomeHandler(object sender, bool status);

Delegaty to szablony metod. Przypisanie metody do obsługi zdarzenia musi posiadać taką samą sygnaturę co delegat tzn. posiadać ten sam zwracany typ oraz te same argumenty.

## Typy generyczne

Deklaracja klasy generycznej wygląda bardzo podobnie do zwykłej klasy, ale posiada
dodatkowe argumenty. Klasa generyczna jest odpowiednikiem szablonu w c++.

In [1]:
public class SimpleClass<T>
{
   public T Field; 
   public T Property { get; set; }
   
   public void DoSomething(T field)
   {
      Field = field;
   }
}

## Ograniczenia argumentu generycznego

Projektując klasy możemy wymusić typ argumentu dla klasy generycznej. Można zdefiniować następujące ograniczenia:

* `T`: `struct` - argument musi być typem prostym,
* `T` : `class` - argumentem musi być klasa,
* `T` : `new()` - argument musi posiadać konstruktor bezparametrowy,
* `T` : `base_class_name` - argument musi dziedziczyć po *base_class_name*,
* `T` : `interface_name` - argument musi implementować *interface_name*,
* `T` : `U` - argument `T` musi dziedziczyć po argumencie `U`.

In [1]:
public class GenericClass<T> where T : Student

Error: [object Object]

## Indeksery

Każda klasa może posiadać tzw. indekser. Użycie indeksera przypomina użycie tablicy. Przykładowy indekser przedstawia poniższy listing:

In [1]:
public T this[int i]
{
   get
   {            
   }
   set
   {            
   }
}
...
IntCollection<int> col = new IntCollection<int>();
col[0] = 12;

Error: [object Object]

## *Tuple*

W przypadku, gdy funkcje zawierają więcej, niż jedną zmienną można utworzyć dedykowaną klasę lub krotkę (ang. *tuple*) w wersji nazwanej (z podanymi nazwami pól w deklaracji) lub nienazwanej (kolejne pola nazywają się *`item1`*, *`item2`* itd).

In [ ]:
public (string name, string surname) tuple_function() {
    return ("Jan", "Kowalski");
}

var person = tuple_function(); 

Console.WriteLine($"Person 1: {person.name} {person.surname}");

(string name, string surname) = tuple_function(); //unboxing

Console.WriteLine($"Person 2: {name} {surname}");

Person 1: Jan Kowalski
Person 2: Jan Kowalski


## Rekord

Rekordy postały z myślą o nowych mechanizmach języka. Pod względem kopiowania (przypisywania) zachowują się jak struktury, ale są typami referencyjnymi. Poniżej znajdują się przykładowe operacje na tych typach niemożliwe do użycia z klasami czy strukturami.

In [ ]:
public record StudentRecord 
{ 
    public string FirstName { get; init; } 
    public string LastName { get; init; }
    public int Id { get; init; }
    public StudentRecord(string firstName, string lastName, int id) 
      => (FirstName, LastName, Id) = (firstName, lastName, id);
    public void Deconstruct(out string firstName, out string lastName, out int id) // jak ma się zachowywać klasa w przypadku dekonstrukcji
      => (firstName, lastName, id) = (FirstName, LastName, Id);
}

public record StudentRecord2(string FirstName, string LastName, int No); //tożsame z powyższym

var student = new StudentRecord("Mads", "Torgersen", 1);
var (f, l, id) = student; 

var student2 = student with { LastName="Kowalski", Id=2 };

Console.WriteLine(student);
Console.WriteLine(student2);

StudentRecord { FirstName = Mads, LastName = Torgersen, Id = 1 }
StudentRecord { FirstName = Mads, LastName = Kowalski, Id = 2 }


## Przydatne techniki wbudowane w język

Operując na plikach należy bezwzględnie zwalniać systemowy uchwyt do pliku metodą `Close()`. Nie wykonanie tej czynności powoduje, że pliku nie da się skasować oraz nadpisać. W przypadku, gdy klasa posiada zaimplementowaną metodę `Dispose()` można użyć konstrukcji:

In [1]:
using (StreamWriter outFile = new StreamWriter(path))
{
   outFile.WriteLine("lancuch");
}

Error: [object Object]

W powyższym listingu nie ma konieczności wywoływania metody `Close()` ponieważ zajmie się tym metoda `Dispose` wywołana niejawnie przy zakończeniu bloku "\}". Powyższy zapis jest równoważny:

In [1]:
try
{
  StreamWriter outFile = new StreamWriter(path);
  outFile.WriteLine("string data");  
}
finally
{
  if(outFile != null)
    outFile.Dispose();
}

Error: [object Object]

## Interpolacja łańcuchów znaków c.d.

Jedną z nowych funkcji w specyfikacji języka jest technika interpolacji łańcuchów znaków, bardzo często spotykana we nowoczesnych językach programowania, takich jak *Python*. Zamiast używać operatora plus do łączenia napisów, programista może sformatować napis. W poprzednim pliku temat został już omówiony, w tym miejscu zostały jedynie dodane inne operacje formatowania. 

In [1]:
int value = 1;
double value2 = 2.12345;

Console.WriteLine($"Integer {value}, double: {value2} and double formated one: {value2:0.0}");
var date = DateTime.Now;
Console.WriteLine($"Now is: {date:HH:mm}")

Integer 1, double: 2,12345 and double formated one: 2,1


Now is: 13:21


## Przydatne klasy

Najczęściej wykorzystywanymi klasami są lista (`List`, `Queue`, `Array`). Przykład użycia listy:

In [1]:
//lista
List<Int32> list = new List<int>();
list.Add(12);
//kolejka
Queue<Int32> queue = new Queue<int>();
queue.Enqueue(12);
Int32 last = queue.Dequeue();

### Klasa `ConfigurationManager`

Często spotykanym podejściem do tworzenia oprogramowania jest uzależnienie wykonania programu od globalnych parametrów wejściowych. Jest to bardzo wygodne podeście ze względu na cykl życia aplikacji. Przykładowo, gdy program łączy się z bazą danych, na etapie testowania połączenie powinno odbywać się z instancją deweloperską bazą danych. Z kolei, gdy program zostanie wdrożony na środowisko produkcyjne, program powinien łączyć się z instancją produkcyjną bazy danych. W środowisku `.NET` powstała specjalna klasa `ConfigurationManager`, która umożliwia w łatwy sposób korzystanie z pliku konfiguracyjnego w formacie `XML` `App.config`. Po etapie kompilacji projektu konfiguracja zostanie przeniesiona do pliku `plik.exe.config`. Klasa `ConfigurationManager` dostępna jest po dodaniu referencji `System.Configuration` do projektu. Można to zrobić klikając prawym przyciskiem myszy na projekt i wybranie opcji `Add` `Reference...`. Poniższy listing przedstawia użycie klasy:

In [1]:
public static void Main(string[] args)
{  
  var strCount = ConfigurationManager.AppSettings["Count"];
  var count = int.Parse(strCount);
}

Error: [object Object]

Przy takiej konfiguracji plik `App.config` powinien wyglądać następująco:

In [1]:
<?xml version="1.0" encoding="utf-8" ?>
<configuration>
  <appSettings>
    <add key="Count" value="20"/>
  </appSettings>
</configuration>

### Klasa `StringBuilder`
W przypadku konkatenacji wielu łańcuchów znaków należy używać klasy `StringBuilder`:

In [1]:
StringBuilder builder = new StringBuilder();
   
foreach(var person in collection)
{
   builder.Append(person.Name);
}      

Error: [object Object]

### Funkcja `Format`
Formatowanie tekstu to jedno z najczęstszych zadań programistycznych. Z tego powodu na platformie `.NET` udostępniono metodę `format`. Jest ona zaimplementowana zarówno w klasie `StringBuilder` w metodzie `AppendFormat()` jak i `string.Format()`:

In [ ]:
//wyswietlenie
string.Format("First argument {0}, drugi {1}"), 1,2);
//zaokraglenie
string.Format("First argument {0:0.00}"), 1.255555);
//daty
string.Format("{0:yyyy-MM-dd}"), DateTime.Now());
//dopelnienie ciagu znakow
string.Format("{0,22}", "Znaki");
string.Format("{0,22}", "Znaki_Znaki");

Error: (2,47): error CS1002: Oczekiwano średnika (;)
(2,47): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(2,50): error CS1002: Oczekiwano średnika (;)
(2,50): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(2,52): error CS1002: Oczekiwano średnika (;)
(2,52): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(4,41): error CS1002: Oczekiwano średnika (;)
(4,41): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(4,51): error CS1002: Oczekiwano średnika (;)
(4,51): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(6,32): error CS1002: Oczekiwano średnika (;)
(6,32): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(6,48): error CS1002: Oczekiwano średnika (;)
(6,48): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku

## Zadania do wykonania

### Zadanie 1

Zaprojektuj klasę abstrakcyjną / rekord, która wyznacza pole, obwód oraz zwraca długości boków trójkąta (dowolnego). Klasami dziedziczącymi z tej klasy ma być trójkąt:
* równoboczny (konstruktor z jednym parametrem),
* równoramienny (konstruktor z dwoma parametrami - dwa różne boki),
* prostokątny (konstruktor z z dwoma parametrami - długości przyprostokątne).

> Przeciąż metodę `ToString()`, która ma zawierać zestaw podstawowych informacji. Zadanie można zrealizować na dwa sposoby: 
1. metody do obliczeń pola i obwodu zaimplementować w klasie abstrakcyjnej używając wzoru:

$
	P =\sqrt{p\cdot(p-A)\cdot(p-B)\cdot(p-C)}
$

gdzie: $p$ to połowa obwodu trójkąta. Symbole $A$, $B$, $C$ to długości poszczególnych boków. Oznaczyć metody obliczające pole i obwód jako abstrakcyjne i przeciążyć je w klasie szczegółowej.

Metoda `ToString` powinna być przeciążona w klasie abstrakcyjnej.

In [2]:
public abstract class Triangle
{
    public double sideA;
    public double SideA
    {
        get { return sideA; }
        set { sideA = value; }
    }
    public double sideB;
    public double SideB
    {
        get { return sideB; }
        set { sideB = value; }
    }
    public double sideC;
    public double SideC
    {
        get { return sideC; }
        set { sideC = value; }
    }
    public double area()
    {
        double p = circuit()/2;
        return Math.Sqrt( p * (p - sideA) * (p - sideB) * (p - sideC) );
    }
    public double circuit()
    {
        return sideA+sideB+sideC;
    }
    public override string ToString()
    {
        return ($"\nSideA: {sideA}\nSideB: {sideB}\nSideC: {sideC}\nArea: {area()}\nCircuit: {circuit()}");
    }
}
public class Equilateral : Triangle  //trojkat rownoboczny
{
    public Equilateral(double sideA)
    {
        SideA = sideA;
        SideB = sideA;
        SideC = sideA;
    }
    ~Equilateral(){}
}
public class Isosceles : Triangle  //trojkat rownoramienny
{
    public Isosceles(double sideA, double sideB)
    {
        SideA = sideA;
        SideB = sideB;
        SideC = sideB;
    }
    ~Isosceles(){}
}
public class Rectangular : Triangle  //trojkat prostokatny
{
    public Rectangular(double sideA, double sideB)
    {
        SideA = sideA;
        SideB = sideB;
        SideC = Math.Sqrt(Math.Pow(sideA, 2) + Math.Pow(sideB, 2));
    }
    ~Rectangular(){}
}

Equilateral eqo = new Equilateral(3);
Isosceles iso = new Isosceles(5, 7);
Rectangular rec = new Rectangular(6, 8);
Console.WriteLine(eqo.ToString());
Console.WriteLine(iso.ToString());
Console.WriteLine(rec.ToString());


SideA: 3
SideB: 3
SideC: 3
Area: 3,897114317029974
Circuit: 9

SideA: 5
SideB: 7
SideC: 7
Area: 16,345871038277526
Circuit: 19

SideA: 6
SideB: 8
SideC: 10
Area: 24
Circuit: 24


### Zadanie 2

Utwórz klasę generyczną kartoteka pracowników, która umożliwia:

* dodawanie/usuwanie,
* wyświetlanie,
* walidację istniejących już pracowników,
* wyszukiwanie danych. 

Klasa pracownik musi zawierać przynajmniej 5 cech. Każdy pracownik powinien posiadać stanowisko pracy. Klasa powinna zawierać walidację danych (poprzez metodę `Validate()`). Klasa pracownik powinna zawierać metody `Show()` i `IsMatch()`, z których będzie korzystała kartoteka przy wyszukiwaniu.

Przy wyświetlaniu pomocna może okazać się metoda `Format()`.

In [31]:
public class Employee {   
    public string employeeId {get; set; }
    public string firstName { get; set; }
    public string lastName { get; set; }
    public string hireDate { get; set; }
    public double salary { get; set; }
    public string workplace { get; set; }
    
    public Employee(string employeeId, string firstName, string lastName, string hireDate, double salary, string workplace) {
        this.employeeId = employeeId;
        this.firstName = firstName;
        this.lastName = lastName;
        this.hireDate = hireDate;
        this.salary = salary;
        this.workplace = workplace;
    }

    public void Validate() {
        if (String.IsNullOrEmpty(employeeId) || String.IsNullOrEmpty(firstName) || String.IsNullOrEmpty(lastName) || String.IsNullOrEmpty(hireDate) || String.IsNullOrEmpty(workplace)) 
        {
           throw new Exception("Incorrect");
        }
        else if (salary <= 0) 
        {
            throw new Exception("Incorrect");
        }
    }

    public string Show() {
        return String.Format(("{0}, {1}, {2}, {3}, {4}, {5}"), employeeId, firstName, lastName, hireDate, salary, workplace);
    }

    public bool IsMatch(string search)
    {
        return search.Equals(employeeId) || search.Equals(firstName) || search.Equals(lastName) || search.Equals(hireDate) || search.Equals(workplace)  ? true : false;
    }

    public bool IsMatch(Employee emp)
    {
        return employeeId.Equals(emp.employeeId)  ? true : false;
    }
}

public class EmployeesCard<T> where T : Employee {
    private List<T> _list = new List<T>();

    public EmployeesCard() {}

    public void Add(T t) {
        _list.Add(t);
    }

    public void Remove(T t) {
        _list.Remove(t);
    }

    public void Show()
    {
        _list.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public void Show(T t)
    {
        Console.WriteLine(t.Show());
    }

    public void Show(List<T> t)
    {
        t.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public bool ValidateExist(T t)
    {
        foreach (var item in _list)
        {
            if (item.IsMatch(t))
            {
                return true;
            }
        }
        return false;
    }

    public T Search(string search)
    {
        foreach (T item in _list)
        {
            if (item.IsMatch(search))
            {
                return item;
            }
        }
        return null;
    }
}


EmployeesCard<Employee> department = new EmployeesCard<Employee>();
department.Add(new Employee("23456", "Wiktor", "Materla", "12-05-2007", 3800, "AD_CDEF"));
department.Add(new Employee("34567", "Karol", "Nowak", "13-05-2007", 4200, "AC_CDEF"));
department.Add(new Employee("45678", "Matylda", "Kowalczyk", "12-05-2007", 5500, "AB_CDEF"));
department.Show();

23456, Wiktor, Materla, 12-05-2007, 3800, AD_CDEF
34567, Karol, Nowak, 13-05-2007, 4200, AC_CDEF
45678, Matylda, Kowalczyk, 12-05-2007, 5500, AB_CDEF


### Zadanie 3

Dodaj do kartoteki z pierwszego zadania odczytywanie i zapisywanie danych w formacie TXT, XML oraz JSON (minimum jeden). Wykorzystaj do tego celu wzorzec projektowy budowniczy. Abstrakcję konkretnego zapisu można przekazać do konstruktora kartoteki, która posiada metodę `Save()`.

In [41]:
using System;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.IO;

public interface IBuilder
{
    string file_type { get; set; }
    string f_path { get; set; }

    void CreateFile(string path);
    string ParseData(Employee emp);
}

public class JsonBuilder : IBuilder
{
    public string file_type { get; set; } = ".json";
    public string f_path { get; set; } = "";

    public void CreateFile(string path)
    {
        f_path = path + file_type;
        StreamWriter sw = File.CreateText(f_path);
        sw.Close();
    }

    public string ParseData(Employee emp)
    {
        return JsonSerializer.Serialize(emp);
    }
}

public class Employee
{
    public string employeeId {get; set; }
    public string firstName { get; set; }
    public string lastName { get; set; }
    public string hireDate { get; set; }
    public double salary { get; set; }
    public string workplace { get; set; }
    
    public Employee(string employeeId, string firstName, string lastName, string hireDate, double salary, string workplace) {
        this.employeeId = employeeId;
        this.firstName = firstName;
        this.lastName = lastName;
        this.hireDate = hireDate;
        this.salary = salary;
        this.workplace = workplace;
    }

    public void Validate() {
        if (String.IsNullOrEmpty(employeeId) || String.IsNullOrEmpty(firstName) || String.IsNullOrEmpty(lastName) || String.IsNullOrEmpty(hireDate) || String.IsNullOrEmpty(workplace)) 
        {
           throw new Exception("Incorrect");
        }
        else if (salary <= 0) 
        {
            throw new Exception("Incorrect");
        }
    }

    public string Show() {
        return String.Format(("{0}, {1}, {2}, {3}, {4}, {5}"), employeeId, firstName, lastName, hireDate, salary, workplace);
    }

    public bool IsMatch(string search)
    {
        return search.Equals(employeeId) || search.Equals(firstName) || search.Equals(lastName) || search.Equals(hireDate) || search.Equals(workplace)  ? true : false;
    }

    public bool IsMatch(Employee emp)
    {
        return employeeId.Equals(emp.employeeId)  ? true : false;
    }
}

public class EmployeesCard<T> where T : Employee {
    private IBuilder ib;
    private List<T> _list = new List<T>();

    public EmployeesCard() {}
    public EmployeesCard(IBuilder ib) {
        this.ib = ib;
    }

    public void Add(T t) {
        _list.Add(t);
    }

    public void Remove(T t) {
        _list.Remove(t);
    }

    public void Show()
    {
        _list.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public void Show(T t)
    {
        Console.WriteLine(t.Show());
    }

    public void Show(List<T> t)
    {
        t.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public bool ValidateExist(T t)
    {
        foreach (var item in _list)
        {
            if (item.IsMatch(t))
            {
                return true;
            }
        }
        return false;
    }

    public T Search(string search)
    {
        foreach (T item in _list)
        {
            if (item.IsMatch(search))
            {
                return item;
            }
        }
        return null;
    }
    
        public void Save(){
        Save("data");
    }

    public void Save(string file_name)
    {
        ib.CreateFile(file_name);
        StreamWriter sw = new StreamWriter(ib.f_path, false);
        
        foreach (var item in _list)
        {
            sw.WriteLine(ib.ParseData(item));
        }

        sw.Close();
    }

    public void Read()
    {
        StreamReader sr = File.OpenText(ib.f_path);
        while (sr.Peek() >= 0)
        {
            Console.WriteLine(sr.ReadLine());
        }
        sr.Close();
    }
}


try {
    EmployeesCard<Employee> department = new EmployeesCard<Employee>(new JsonBuilder());
    department.Add(new Employee("23456", "Wiktor", "Materla", "12-05-2007", 3800, "AD_CDEF"));
    department.Add(new Employee("34567", "Karol", "Nowak", "13-05-2007", 4200, "AC_CDEF"));
    department.Add(new Employee("45678", "Matylda", "Kowalczyk", "12-05-2007", 5500, "AB_CDEF"));
    department.Show();
    department.Save();
    department.Read();
}
catch (Exception e) {
    Console.WriteLine(e);
}

23456, Wiktor, Materla, 12-05-2007, 3800, AD_CDEF
34567, Karol, Nowak, 13-05-2007, 4200, AC_CDEF
45678, Matylda, Kowalczyk, 12-05-2007, 5500, AB_CDEF
{"employeeId":"23456","firstName":"Wiktor","lastName":"Materla","hireDate":"12-05-2007","salary":3800,"workplace":"AD_CDEF"}
{"employeeId":"34567","firstName":"Karol","lastName":"Nowak","hireDate":"13-05-2007","salary":4200,"workplace":"AC_CDEF"}
{"employeeId":"45678","firstName":"Matylda","lastName":"Kowalczyk","hireDate":"12-05-2007","salary":5500,"workplace":"AB_CDEF"}


### Zadanie 4
 
Rozszerz poprzednie zadanie o możliwość szyfrowania danych metodą cezara. Blok przesunięć powinien być większy, niż jeden. W celu jego przechowywania wykorzystaj klasę `ConfigurationManager` (najpierw należy dodać referencję do biblioteki `System.Configuration`).

In [61]:
using System.Configuration;

using System;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.IO;

public interface IBuilder
{
    string file_type { get; set; }
    string f_path { get; set; }

    void CreateFile(string path);
    string ParseData(Employee emp);
}

public class JsonBuilder : IBuilder
{
    public string file_type { get; set; } = ".json";
    public string f_path { get; set; } = "";

    public void CreateFile(string path)
    {
        f_path = path + file_type;
        StreamWriter sw = File.CreateText(f_path);
        sw.Close();
    }

    public string ParseData(Employee emp)
    {
        return JsonSerializer.Serialize(emp);
    }
}

public class Employee
{
    public string employeeId {get; set; }
    public string firstName { get; set; }
    public string lastName { get; set; }
    public string hireDate { get; set; }
    public double salary { get; set; }
    public string workplace { get; set; }
    
    public Employee(string employeeId, string firstName, string lastName, string hireDate, double salary, string workplace) {
        this.employeeId = employeeId;
        this.firstName = firstName;
        this.lastName = lastName;
        this.hireDate = hireDate;
        this.salary = salary;
        this.workplace = workplace;
    }

    public void Validate() {
        if (String.IsNullOrEmpty(employeeId) || String.IsNullOrEmpty(firstName) || String.IsNullOrEmpty(lastName) || String.IsNullOrEmpty(hireDate) || String.IsNullOrEmpty(workplace)) 
        {
           throw new Exception("Incorrect");
        }
        else if (salary <= 0) 
        {
            throw new Exception("Incorrect");
        }
    }

    public string Show() {
        return String.Format(("{0}, {1}, {2}, {3}, {4}, {5}"), employeeId, firstName, lastName, hireDate, salary, workplace);
    }

    public bool IsMatch(string search)
    {
        return search.Equals(employeeId) || search.Equals(firstName) || search.Equals(lastName) || search.Equals(hireDate) || search.Equals(workplace)  ? true : false;
    }

    public bool IsMatch(Employee emp)
    {
        return employeeId.Equals(emp.employeeId)  ? true : false;
    }
}

public class CaesarShift {
    string strCount = ConfigurationManager.AppSettings["Count"];
    public CaesarShift() {}
    private static char Shift(char ch, int key)
    {
        if (!char.IsLetter(ch))
            return ch;

        var offset = char.IsUpper(ch) ? 'A' : 'a';
        return (char)((((ch + key) - offset) % 26) + offset);
    }

    public string Encode(string input, int key)
    {
        var output = string.Empty;

        foreach (char ch in input)
            output += Shift(ch, key);

        return output;
    }

    public string Decode(string input, int key)
    {
        return Encode(input, 26 - key);
    }
}

public class EmployeesCard<T> where T : Employee {
    private IBuilder ib;
    private List<T> _list = new List<T>();
    CaesarShift cipher = new CaesarShift();

    public EmployeesCard() {}
    public EmployeesCard(IBuilder ib) {
        this.ib = ib;
    }

    public void Add(T t) {
        _list.Add(t);
    }

    public void Remove(T t) {
        _list.Remove(t);
    }

    public void Show()
    {
        _list.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public void Show(T t)
    {
        Console.WriteLine(t.Show());
    }

    public void Show(List<T> t)
    {
        t.ForEach(item => Console.WriteLine("{0}", item.Show()));
    }

    public bool ValidateExist(T t)
    {
        foreach (var item in _list)
        {
            if (item.IsMatch(t))
            {
                return true;
            }
        }
        return false;
    }

    public T Search(string search)
    {
        foreach (T item in _list)
        {
            if (item.IsMatch(search))
            {
                return item;
            }
        }
        return null;
    }
    
        public void Save(){
        Save("data");
    }

    public void Save(string file_name)
    {
        ib.CreateFile(file_name);
        StreamWriter sw = new StreamWriter(ib.f_path, false);
        
        foreach (var item in _list)
        {
            sw.WriteLine((cipher.Encode(ib.ParseData(item), 2)));
        }

        sw.Close();
    }

    public void Read()
    {
        StreamReader sr = File.OpenText(ib.f_path);
        while (sr.Peek() >= 0)
        {
            Console.WriteLine(cipher.Decode(sr.ReadLine(), 2));
        }
        sr.Close();
    }
}


try {
    EmployeesCard<Employee> department = new EmployeesCard<Employee>(new JsonBuilder());
    department.Add(new Employee("23456", "Wiktor", "Materla", "12-05-2007", 3800, "AD_CDEF"));
    department.Add(new Employee("34567", "Karol", "Nowak", "13-05-2007", 4200, "AC_CDEF"));
    department.Add(new Employee("45678", "Matylda", "Kowalczyk", "12-05-2007", 5500, "AB_CDEF"));
    department.Show();
    department.Save();
    department.Read();
}
catch (Exception e) {
    Console.WriteLine(e);
}

try {
    EmployeesCard<Employee> department = new EmployeesCard<Employee>(new JsonBuilder());

    
    department.Add(new Employee("23456", "Wiktor", "Materla", "12-05-2007", 3800, "AD_CDEF"));
    department.Add(new Employee("34567", "Karol", "Nowak", "13-05-2007", 4200, "AC_CDEF"));
    department.Add(new Employee("45678", "Matylda", "Kowalczyk", "12-05-2007", 5500, "AB_CDEF"));
    department.Show();
    department.Save();
    department.Read();
}
catch (Exception e) {
    Console.WriteLine(e);
}

23456, Wiktor, Materla, 12-05-2007, 3800, AD_CDEF
34567, Karol, Nowak, 13-05-2007, 4200, AC_CDEF
45678, Matylda, Kowalczyk, 12-05-2007, 5500, AB_CDEF
{"employeeId":"23456","firstName":"Wiktor","lastName":"Materla","hireDate":"12-05-2007","salary":3800,"workplace":"AD_CDEF"}
{"employeeId":"34567","firstName":"Karol","lastName":"Nowak","hireDate":"13-05-2007","salary":4200,"workplace":"AC_CDEF"}
{"employeeId":"45678","firstName":"Matylda","lastName":"Kowalczyk","hireDate":"12-05-2007","salary":5500,"workplace":"AB_CDEF"}
23456, Wiktor, Materla, 12-05-2007, 3800, AD_CDEF
34567, Karol, Nowak, 13-05-2007, 4200, AC_CDEF
45678, Matylda, Kowalczyk, 12-05-2007, 5500, AB_CDEF
{"employeeId":"23456","firstName":"Wiktor","lastName":"Materla","hireDate":"12-05-2007","salary":3800,"workplace":"AD_CDEF"}
{"employeeId":"34567","firstName":"Karol","lastName":"Nowak","hireDate":"13-05-2007","salary":4200,"workplace":"AC_CDEF"}
{"employeeId":"45678","firstName":"Matylda","lastName":"Kowalczyk","hireDate":